In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
cur_path = "./gdrive/Othercomputers/Desktop_Vivian/twitter_hashtag_recommendor/"
os.chdir(cur_path)
!pwd

/content/gdrive/Othercomputers/Desktop_Vivian/twitter_hashtag_recommendor


In [3]:
!pip install transformers
# !pip install --upgrade torch torchvision

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 41.7 MB/s 
     |████████████████████████████████| 596 kB 55.1 MB/s 
     |████████████████████████████████| 6.6 MB 44.9 MB/s 
     |████████████████████████████████| 77 kB 8.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
! sudo apt-get install git-lfs
! git lfs install

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 6,800 kB of archives.
After this operation, 15.3 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 3.1.2 [6,800 kB]
Fetched 6,800 kB in 1s (13.2 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [5]:
from transformers import BertTokenizer, BertForMaskedLM, BertModel
import torch
print(torch.__version__)

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForMaskedLM.from_pretrained('bert-base-uncased')

1.10.0+cu111


In [6]:
import string as st
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def pre_processing(text, punc):
    texts = text.split()
    a = []
    for t in texts:
      for n in t:
        if n in punc:
          t = t.replace(n, "")
      s = t.lower()
      a.append(s)
    return ' '.join(a)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split

labeled_tweet = pd.read_csv("./data/labeled_tweet2.csv")
labeled_text = [str(s) for s in labeled_tweet['cleaned_tweet'].tolist()]

punc = '''!"@$%&\'()*+,-./:;<=>?\\^_`{|}[]~'''
labeled_text = [pre_processing(t, punc) for t in labeled_text]

train_text, test_text = train_test_split(labeled_text, random_state=2018, test_size=0.2)


## Add special tokens (hashtags)

In [41]:
special_tokens = {}
for t in labeled_text:
  for s in t.split():
    if s.startswith('#'):
      if s in special_tokens:
        special_tokens[s] += 1
      else:
        special_tokens[s] = 1
import operator
sp_tks = list(dict(sorted(special_tokens.items(), key=operator.itemgetter(1),reverse=True)).keys())[:1000]

In [58]:
pd.DataFrame(sp_tks).to_csv("./special_tokens.csv", header=False)

In [42]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', additional_special_tokens=sp_tks)
# tokenizer.save_pretrained('./bert_twitter_hashtag/')
# model = BertModel.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.resize_token_embeddings(len(tokenizer))

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7

Embedding(31521, 768)

In [43]:


train_inputs = tokenizer(train_text, return_tensors='pt', max_length=128, truncation=True, padding='max_length')
train_inputs['labels'] = train_inputs.input_ids.detach().clone()

test_inputs = tokenizer(test_text, return_tensors='pt', max_length=128, truncation=True, padding='max_length')
test_inputs['labels'] = test_inputs.input_ids.detach().clone()

In [45]:
for i in range(len(test_inputs)):
  for j in range(128):
    test_inputs['attention_mask'][i][j] = 0

for i in range(len(test_text)):
  for t in test_text[i].split():

    if t in sp_tks:
      tks = tokenizer.encode(t, add_special_tokens=False)[0]

      test_inputs['attention_mask'][i][test_inputs['input_ids'][i].tolist().index(tks)] = 1


In [48]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)


train_dataset = MeditationsDataset(train_inputs)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = MeditationsDataset(test_inputs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)

In [49]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31521, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
           

In [50]:
from transformers import AdamW
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [13]:
from tqdm import tqdm  # for our progress bar

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/1553 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 1: 100%|██████████| 1553/1553 [06:34<00:00,  3.94it/s, loss=9.12e-5]


In [51]:
from transformers import TrainingArguments

batch_size = 16
training_args = TrainingArguments(
    output_dir='bert_twitter_hashtag',
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    num_train_epochs=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size
    # push_to_hub=True,
    # fp16=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [20]:
! huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machin

In [52]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [53]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 24844
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3106
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Epoch,Training Loss,Validation Loss
1,0.000400,0.002775
2,0.000200,0.002858


Saving model checkpoint to bert_twitter_hashtag/checkpoint-500
Configuration saved in bert_twitter_hashtag/checkpoint-500/config.json
Model weights saved in bert_twitter_hashtag/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Saving model checkpoint to bert_twitter_hashtag/checkpoint-1000
Configuration saved in bert_twitter_hashtag/checkpoint-1000/config.json
Model weights saved in bert_twitter_hashtag/checkpoint-1000/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Saving model checkpoint to bert_twitter_has

TrainOutput(global_step=3106, training_loss=0.06326158193563138, metrics={'train_runtime': 913.0388, 'train_samples_per_second': 54.42, 'train_steps_per_second': 3.402, 'total_flos': 3269568306825216.0, 'train_loss': 0.06326158193563138, 'epoch': 2.0})

In [23]:
import math

eval_results = trainer.evaluate()
eval_results

***** Running Evaluation *****
  Num examples = 6211
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


{'epoch': 2.0,
 'eval_loss': 9.91832057479769e-05,
 'eval_runtime': 30.9988,
 'eval_samples_per_second': 200.362,
 'eval_steps_per_second': 12.549}

In [54]:
model.save_pretrained('./bert_twitter_hashtag/')

Configuration saved in ./bert_twitter_hashtag/config.json
Model weights saved in ./bert_twitter_hashtag/pytorch_model.bin


In [55]:
tokenizer.save_pretrained('./bert_twitter_hashtag/')

tokenizer config file saved in ./bert_twitter_hashtag/tokenizer_config.json
Special tokens file saved in ./bert_twitter_hashtag/special_tokens_map.json
added tokens file saved in ./bert_twitter_hashtag/added_tokens.json


('./bert_twitter_hashtag/tokenizer_config.json',
 './bert_twitter_hashtag/special_tokens_map.json',
 './bert_twitter_hashtag/vocab.txt',
 './bert_twitter_hashtag/added_tokens.json')

In [39]:
# trainer.push_to_hub()

Saving model checkpoint to bert_twitter_hashtag
Configuration saved in bert_twitter_hashtag/config.json
Model weights saved in bert_twitter_hashtag/pytorch_model.bin
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


KeyboardInterrupt: ignored